<a href="https://colab.research.google.com/github/RZx-Taishou/Facial-Pain-Recognition/blob/main/Facial_Pain_Recognition_Kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet152V2
import sklearn
from sklearn.model_selection import KFold, train_test_split
import pathlib
import os
import numpy as np
import pandas as pd

In [31]:
!git clone https://github.com/RZx-Taishou/Facial-Pain-Recognition.git

fatal: destination path 'Facial-Pain-Recognition' already exists and is not an empty directory.


In [32]:
ORIGINAL_DIR = "/content/Facial-Pain-Recognition/Dataset"

In [33]:
#dataframe
data_path = pathlib.Path(ORIGINAL_DIR)

img_path = list(data_path.glob('**/*.*'))

img_labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], img_path))

pd_img_path = pd.Series(img_path, name='PATH').astype(str)
pd_img_labels = pd.Series(img_labels, name='LABELS').astype(str)

img_df = pd.merge(pd_img_path, pd_img_labels, right_index=True, left_index=True)

img_df = img_df.sample(frac = 1).reset_index(drop=True)
img_df.head()

,PATH,LABELS
0,/content/Facial-Pain-Recognition/Dataset/Neutr...,Neutral
1,/content/Facial-Pain-Recognition/Dataset/Neutr...,Neutral
2,/content/Facial-Pain-Recognition/Dataset/Pain/...,Pain
3,/content/Facial-Pain-Recognition/Dataset/Pain/...,Pain
4,/content/Facial-Pain-Recognition/Dataset/Pain/...,Pain


import numpy as np
from google.colab import autoviz
df_5946474719986938405 = autoviz.get_df('df_5946474719986938405')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_5946474719986938405, *['PATH'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5946474719986938405 = autoviz.get_df('df_5946474719986938405')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_5946474719986938405, *['LABELS'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5946474719986938405 = autoviz.get_df('df_5946474719986938405')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_5946474719986938405, *['PATH', 'LABELS'], **{})
chart

In [44]:
img_df['LABELS'].value_counts(ascending=True)
# It is small dataset

Neutral    244
Pain       652
Name: LABELS, dtype: int64

In [34]:
#Split Data
train_dataset, test_dataset = train_test_split(img_df, train_size=0.8, shuffle=True, stratify=img_df['LABELS'])
print("Number of train data:", train_dataset.shape[0])
print("Number of test data:", test_dataset.shape[0])

Number of train data: 716
Number of test data: 180


In [35]:
#imagedatagenerator
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [36]:
effb7 = ResNet152V2(input_shape=(150, 150, 3), include_top=False, weights="imagenet")

for layer in effb7.layers:
  layer.trainable = False

x = effb7.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(.2)(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = Model(effb7.input, x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 77, 77, 64)   0           ['conv1_conv[0][0]']       

In [37]:
loss = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True,
    alpha=0.25,
    gamma=2.0,
    from_logits=False,
)

In [38]:
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [55]:
#kfold
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
for fold, (train_index, val_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold + 1}:")

    train_generator = datagen.flow_from_dataframe(
        img_df.loc[train_index],
        x_col='PATH', y_col='LABELS',
        target_size=(150, 150),
        class_mode="binary",
        batch_size=64,
        shuffle=True
        #subset="training"
)

    validation_generator = datagen.flow_from_dataframe(
        img_df.loc[val_index],
        x_col='PATH', y_col='LABELS',
        target_size=(150, 150),
        class_mode="binary",
        batch_size=32,
        shuffle=True
        #subset="validation"
)

    model.fit(train_generator, epochs=10, validation_data=validation_generator,verbose=2)

Fold 1:
Found 572 validated image filenames belonging to 2 classes.
Found 144 validated image filenames belonging to 2 classes.
Epoch 1/10
9/9 - 50s - loss: 0.6910 - accuracy: 0.5874 - val_loss: 0.2196 - val_accuracy: 0.5694 - 50s/epoch - 6s/step
Epoch 2/10
9/9 - 42s - loss: 0.1919 - accuracy: 0.6818 - val_loss: 0.0938 - val_accuracy: 0.6111 - 42s/epoch - 5s/step
Epoch 3/10
9/9 - 43s - loss: 0.0850 - accuracy: 0.6591 - val_loss: 0.0560 - val_accuracy: 0.7778 - 43s/epoch - 5s/step
Epoch 4/10
9/9 - 42s - loss: 0.0620 - accuracy: 0.7552 - val_loss: 0.0568 - val_accuracy: 0.7917 - 42s/epoch - 5s/step
Epoch 5/10
9/9 - 45s - loss: 0.0608 - accuracy: 0.7640 - val_loss: 0.0503 - val_accuracy: 0.6736 - 45s/epoch - 5s/step
Epoch 6/10
9/9 - 42s - loss: 0.0553 - accuracy: 0.6241 - val_loss: 0.0466 - val_accuracy: 0.6667 - 42s/epoch - 5s/step
Epoch 7/10


In [ ]:
model.save("FPR.h5")

In [ ]:
from google.colab import files
files.download("FPR.h5")

In [15]:
model.fit(train_generator, epochs=100, validation_data=validation_generator, verbose=2)

KeyboardInterrupt: ignored